<a href="https://colab.research.google.com/github/kuzminadya/FHP-infovis-py/blob/master/Data_Dossier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Dossier | Consultation on 11 June

## An idea

My idea is to analyse and visualise the transformation of font file changing over time. Creating a font, especially a large font family may take several years of work or it might be a result of collaboration of several designers and type engineers. Even after the first version is released, the contribution to the project and updating the font may last for several years.

*add text about version control system*

## Title (the first version)

Visualising font file transformation over time 

or

Visualisation and analysis of font file transformation over time 

or

As if the font was software. Visualization of the font development process

## An approximate work plan

[x] Choose a topic and/or dataset you are interested in\
[x] Prepare the data for the first use case\
[ ] Prepare the data for the second use case → solve issues till 25.06.2020\
[x] Pose a question to apply your visualization skills\
[ ] Generate several visualizations (static and interactive) → 25.06.2020\
[ ] Include the most informative in report on your process → 02.07.2020\
[ ] Share the resulting report as a Jupyter Notebook → 02.02.2020


## Use cases

There are two possible ways of elaborating my experiment. I can visualise either the structure of the existing font (the first use case) or I can create one, using UFO format (the second use case). For the second story I need to prepare a sample dataset myself.

### The first use case

As an examle I use Source Code Pro Project from Adobe, which took part for 4 years:\
https://github.com/adobe-fonts/source-code-pro/commits/release

And IBM Plex® typeface which was active on Github for a 2 years and have 219 commits:\
https://github.com/IBM/plex 

### The second use case

For the second project I decided to use my own current project [Mogee font](http://nadyakuzmina.com/story-of-mogeefont)

The font originally was made in glyphs and I need to prepare all the existing versions in UFO format.

https://robofont.com/documentation/how-tos/using-git/

> The UFO file format is perfect for version control, since it is text-based. A UFO is basically just a folder with many little XML-files in it. A UFO file could theoretically be created and modified in a text editor. This opens the field of digital type design for version control tools normally used only by software developers.

## IBM Plex®

Question

https://github.com/IBM/plex/blob/master/LICENSE.txt


In [0]:
import pandas as pd
import altair as alt

The first step is getting the dataset in JSON format from Github. It includes two files: a description of the repo and a list of commits.

https://developer.github.com/v3/repos/#get-a-repository

/repos/:owner/:repo

In [19]:
import requests 

url = ('https://api.github.com/repos/IBM/plex')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_repo_json = response.json()

# dataset_repo_json

Request returned 200 : 'OK'


https://developer.github.com/v3/repos/commits/ 

GET /repos/:owner/:repo/commits

In [20]:
url = ('https://api.github.com/repos/IBM/plex/commits')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_commits_json = response.json()

# dataset_commits_json

Request returned 200 : 'OK'


The next part is converting JSON into table.

https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [21]:
from pandas.io.json import json_normalize
df_commits_table = pd.DataFrame.from_dict(json_normalize(dataset_commits_json), orient='columns')
df_repo_table = pd.DataFrame.from_dict(json_normalize(dataset_repo_json), orient='columns')
# df_commits_table
# df_repo_table

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


Then hiding unnecessary columns

https://colab.research.google.com/drive/11zDWTJHup4A4rEUZq2evGVpXagElfTdT?usp=sharing

In [22]:
df_commits_table_reduced = df_commits_table.drop(columns=["sha", "node_id", "parents", "commit.author.email", "commit.committer.email", "commit.tree.sha", "commit.verification.payload",  "author.login", "author.id", "author.node_id", "author.gravatar_id", "committer.id", "committer.node_id", "committer.gravatar_id"])

df_commits_table_reduced.sample(5)

,url,html_url,comments_url,commit.author.name,commit.author.date,commit.committer.name,commit.committer.date,commit.message,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,author.avatar_url,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.avatar_url,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,author,committer
18,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/2224b57a801...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-10-03T18:45:57Z,Vince Picone,2019-10-03T18:45:57Z,breaking: update thai and devanagari typeface ...,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/877cd331a70...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-11-18T16:03:37Z,Vince Picone,2019-11-18T16:03:37Z,4.0.0,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,False,unsigned,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/549de9c7f98...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-11-18T16:30:10Z,GitHub,2019-11-18T16:30:10Z,Update CHANGELOG.md,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,https://avatars1.githubusercontent.com/u/40780...,https://api.github.com/users/vpicone,https://github.com/vpicone,https://api.github.com/users/vpicone/followers,https://api.github.com/users/vpicone/following...,https://api.github.com/users/vpicone/gists{/gi...,https://api.github.com/users/vpicone/starred{/...,https://api.github.com/users/vpicone/subscript...,https://api.github.com/users/vpicone/orgs,https://api.github.com/users/vpicone/repos,https://api.github.com/users/vpicone/events{/p...,https://api.github.com/users/vpicone/received_...,User,False,web-flow,https://avatars3.githubusercontent.com/u/19864...,https://api.github.com/users/web-flow,https://github.com/web-flow,https://api.github.com/users/web-flow/followers,https://api.github.com/users/web-flow/followin...,https://api.github.com/users/web-flow/gists{/g...,https://api.github.com/users/web-flow/starred{...,https://api.github.com/users/web-flow/subscrip...,https://api.github.com/users/web-flow/orgs,https://api.github.com/users/web-flow/repos,https://api.github.com/users/web-flow/events{/...,https://api.github.com/users/web-flow/received...,User,False,NaN,NaN
21,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/ad27880ae51...,https://api.github.com/repos/IBM/plex/commits/...,Vince Picone,2019-06-04T19:00:48Z,GitHub,2019-06-04T19:00:48Z,Thai looped update (#256)\n\nThai looped update,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,https://avatars1.githubusercontent.com/u/40780...,https://api.github.com/users/vpicone,https://github.com/vpicone,https://api.github.com/users/vpicone/followers,https://api.github.com/users

In [23]:
df_repo_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 115 entries, id to organization.site_admin
dtypes: bool(11), int64(13), object(91)
memory usage: 971.0+ bytes


In [24]:
df_commits_table_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 45 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            30 non-null     object 
 1   html_url                       30 non-null     object 
 2   comments_url                   30 non-null     object 
 3   commit.author.name             30 non-null     object 
 4   commit.author.date             30 non-null     object 
 5   commit.committer.name          30 non-null     object 
 6   commit.committer.date          30 non-null     object 
 7   commit.message                 30 non-null     object 
 8   commit.tree.url                30 non-null     object 
 9   commit.url                     30 non-null     object 
 10  commit.comment_count           30 non-null     int64  
 11  commit.verification.verified   30 non-null     bool   
 12  commit.verification.reason     30 non-null     objec

Then parsing *commit.committer.date          30 non-null     object* in datetime format


In [25]:
df_commits_table_reduced['commit.author.date'] = pd.to_datetime(df_commits_table_reduced['commit.author.date'], format='%Y-%m-%d')

df_commits_table_reduced['commit.author.date'].sample(5)

7    2019-11-18 16:29:14+00:00
12   2019-11-18 15:52:50+00:00
18   2019-10-03 18:45:57+00:00
29   2019-04-05 18:27:02+00:00
5    2019-11-23 11:32:27+00:00
Name: commit.author.date, dtype: datetime64[ns, UTC]

Couinting amount of commits for each day 

In [26]:
df_time_index = df_commits_table_reduced.set_index("commit.author.date")
commits_sums = df_time_index.resample("D").sum()
commits_sums

# ha-ha, doesn't work

,commit.comment_count,commit.verification.verified,author,committer
commit.author.date,,,,
2019-04-05 00:00:00+00:00,0,1.0,0.0,0.0
2019-04-06 00:00:00+00:00,0,0.0,0.0,0.0
2019-04-07 00:00:00+00:00,0,0.0,0.0,0.0
2019-04-08 00:00:00+00:00,0,1.0,0.0,0.0
2019-04-09 00:00:00+00:00,0,0.0,0.0,0.0
...,...,...,...,...
2020-05-08 00:00:00+00:00,0,0.0,0.0,0.0
2020-05-09 00:00:00+00:00,0,0.0,0.0,0.0
2020-05-10 00:00:00+00:00,0,0.0,0.0,0.0


A test visualisation:

it could be a visualisation of amount of commits each day + commit comments on hover

## Second use case

The second use case — analyse and comperison of UFO font files

xml

https://colab.research.google.com/drive/1u4gucw1Lndk_Wpuo3pAhojEUp5iiq19u

https://docs.python.org/2/library/xml.etree.elementtree.html

ufo

http://unifiedfontobject.org/versions/ufo3/

**The question:\
how to iterate through all font files on github?**


In [0]:
import xml.etree.ElementTree as ET
import requests

In order to test it I've upload each lip.plist file from all 3 versions via url 

In [0]:
url_ver3 = 'https://raw.githubusercontent.com/kuzminadya/FHP-infovis-py/master/ufo/2020.04.11mogeefont-v3.1.ufo/lib.plist'

In [29]:
response_ver3 = requests.get(url_ver3)
ufo_xml = response_ver3.text
ufo_xml

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">\n<plist version="1.0">\n<dict>\n\t<key>com.schriftgestaltung.disablesAutomaticAlignment</key>\n\t<true/>\n\t<key>com.schriftgestaltung.font.customParameters</key>\n\t<array>\n\t\t<dict>\n\t\t\t<key>name</key>\n\t\t\t<string>glyphOrder</string>\n\t\t\t<key>value</key>\n\t\t\t<array>\n\t\t\t\t<string>space</string>\n\t\t\t\t<string>exclam</string>\n\t\t\t\t<string>quotesingle</string>\n\t\t\t\t<string>quotedbl</string>\n\t\t\t\t<string>numbersign</string>\n\t\t\t\t<string>dollar</string>\n\t\t\t\t<string>percent</string>\n\t\t\t\t<string>ampersand</string>\n\t\t\t\t<string>parenleft</string>\n\t\t\t\t<string>parenright</string>\n\t\t\t\t<string>asterisk</string>\n\t\t\t\t<string>plus</string>\n\t\t\t\t<string>comma</string>\n\t\t\t\t<string>hyphen</string>\n\t\t\t\t<string>period</string>\n\t\t\t\t<string>slash</string>\n\t\t\t\t<string>

In [31]:
print(ufo_xml)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple Computer//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<key>com.schriftgestaltung.disablesAutomaticAlignment</key>
	<true/>
	<key>com.schriftgestaltung.font.customParameters</key>
	<array>
		<dict>
			<key>name</key>
			<string>glyphOrder</string>
			<key>value</key>
			<array>
				<string>space</string>
				<string>exclam</string>
				<string>quotesingle</string>
				<string>quotedbl</string>
				<string>numbersign</string>
				<string>dollar</string>
				<string>percent</string>
				<string>ampersand</string>
				<string>parenleft</string>
				<string>parenright</string>
				<string>asterisk</string>
				<string>plus</string>
				<string>comma</string>
				<string>hyphen</string>
				<string>period</string>
				<string>slash</string>
				<string>zero</string>
				<string>one</string>
				<string>two</string>
				<string>three</string>
				<string>four</string>

In [30]:
lib-tree = ET.parse(ufo_xml)

SyntaxError: ignored

connect commits + visualise which particular changes has happened?\
what kind of statistic can I aggregate based on the whole font file?\
github: font file structure + commit comment // font file // each glyph separatly
